In [ ]:
def integrate_particles(positions, concentrations, streamfunction_field, X, Y, dt, num_steps, phase_field, gamma, r):
    num_particles = len(positions)
    positions_over_time,  concentrations_over_time = np.zeros((num_steps+1, num_particles, 2)), np.zeros((num_steps+1, num_particles, 1))
    positions_over_time[0] = positions
    concentrations_over_time[0] = concentrations
    #u -(1-gamma)psidy + gamma*phidx
    #v (1-gamma)psidx + gamma*phidy
    #dcdt = gamma * -c * (phixx +phiyy)
    psidy, psidx, phidx, phidy = diff_wrt_y(streamfunction_field), diff_wrt_x(streamfunction_field), diff_wrt_x(phase_field), diff_wrt_y(phase_field)
    phixx, phiyy = diff_wrt_x(phidx), diff_wrt_y(phidy)
    Umat = -(1-gamma)*psidy + gamma*phidx
    Vmat = (1-gamma)*psidx + gamma*phidy
    Cmat = gamma * (phixx + phiyy)

    t = [i*dt for i in range(num_steps+1)]
    polynomial_u, polynomial_v, polynomial_c = interpolate(Umat, X, Y, r), interpolate(Vmat, X, Y, r), interpolate(Cmat, X, Y, r)

    y = np.concatenate((positions_over_time, concentrations_over_time), axis=2)
    answer = new_vector_rk4(fun, t, y, num_particles, polynomial_u, polynomial_v, polynomial_c)

    return answer


def diff_wrt_y(original_matrix): 
    nt = original_matrix.shape[0]
    umatrix = np.copy(original_matrix)
    for i in range(nt):
        umatrix[i] = np.gradient(original_matrix[i], axis=1) 
    #print(umatrix)
    return umatrix

def diff_wrt_x(original_matrix):
    vmatrix = np.copy(original_matrix)
    nt = original_matrix.shape[0]
    for i in range(nt):
        vmatrix[i] = np.gradient(original_matrix[i], axis=0) 
    return vmatrix

def interpolate(matr, X, Y, r):

    t = matr.shape[0]
    time = [i*r for i in range(t)]
    interpolator = RegularGridInterpolator((time, X, Y), np.real(matr), method = "cubic")
    return interpolator

def new_vector_rk4(f, t, y, num_particles, polynomial_u, polynomial_v, polynomial_c):
    n = len(t)
    for i in range(n - 1):
        h = t[i+1] - t[i]
        k1 = f(t[i], (y[i]), num_particles, polynomial_u, polynomial_v, polynomial_c)
        k2 = f(t[i] + 0.5*h, (y[i] + 0.5*h*k1), num_particles, polynomial_u, polynomial_v, polynomial_c)
        k3 = f(t[i] + 0.5*h, (y[i] + 0.5*h*k2), num_particles, polynomial_u, polynomial_v, polynomial_c)
        k4 = f(t[i+1], (y[i] + h*k3), num_particles, polynomial_u, polynomial_v, polynomial_c)
        y[i+1] = y[i] + (h/6)*(k1 + 2*k2 + 2*k3 + k4) #remove moduloL to get the real trajectories, keep it for pseudo
    #print("h below")
    #print(h)
    return y

def moduloL(y):
    L =  np.pi
    for i in range(len(y)): 
        y[i,0] = np.mod(y[i,0] + L, 2 * L) - L
        y[i,1] = np.mod(y[i,1] + L, 2 * L) - L
    return y

def fun(t,y, num_particles, polynomial_u, polynomial_v, polynomial_c):
    y = moduloL(y)
    #print(y)
    xlist, ylist, clist = [y[i,0] for i in range(num_particles)], [y[i,1] for i in range(num_particles)], [y[i,2] for i in range(num_particles)]
    timecoords = t*np.ones(num_particles)
    coords = [[x,y,z] for x,y,z in zip(timecoords, xlist, ylist)]
    dxdt = np.array(polynomial_u(coords)).reshape(-1, 1)
    #print(polynomial_u(coords))
    dydt = np.array(polynomial_v(coords)).reshape(-1, 1)
    resultone = np.concatenate((dxdt, dydt), axis=1)

    c = np.array(clist).reshape(-1,1)
    dcdt = -c * np.array(polynomial_c(coords)).reshape(-1, 1)

    #dcdt = 0 * np.array(polynomial_c(coords)).reshape(-1, 1)

    result = np.concatenate((resultone, dcdt), axis=1)
    return result

               